In [23]:
import torch
from torchvision import models
from pretrained_inceptionv3_alt import pretrained_inception_v3
from custom_dset_new_alt import train_val_test_split, custom_dset
from torch.utils.data import Dataset, DataLoader
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torchvision import models, transforms, utils
from PIL import Image
from tqdm import tqdm
from torch.optim import Adam, SGD, lr_scheduler
import math
from torch.utils.data import WeightedRandomSampler
from collections import Counter
import time
from pretrained_inceptionv3_alt import pretrained_inception_v3
from custom_dset_new_alt import custom_dset, train_val_test_split
import pickle

In [2]:
model_path = '../../data/CNN_model_landtype_alt/().pt'

In [3]:
model = pretrained_inception_v3(6, False)

In [4]:
model.load_state_dict(torch.load(model_path))

In [6]:
from test_alt import test

In [7]:
data_dir = '../../data/images/'

In [18]:
train, val, test = train_val_test_split(data_dir, 0.7, 0.2, 0.1)

In [14]:
model.eval()

pretrained_inception_v3(
  (model): Inception3(
    (Conv2d_1a_3x3): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2a_3x3): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_2b_3x3): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_3b_1x1): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (Conv2d_4a_3x3): BasicConv2d(
      (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=F

In [30]:
model = model.cuda()

In [21]:
dset = custom_dset(data_files = test, transform = 'val')

In [22]:
dataloader = DataLoader(dset, batch_size = 24, shuffle = True, num_workers = 1, pin_memory = False)

In [25]:
loss_fun = torch.nn.CrossEntropyLoss(reduction = 'sum')

In [26]:
running_loss = 0.0
running_corrects = 0
size = 0

In [33]:
# now iterate over the images to make predictions
for inputs, labels in tqdm(dataloader):
    inputs = inputs.cuda()
    labels = labels.cuda()
    # counting how many images is contained in this batch
    batch_count = labels.size(0)
    # Forward pass
    output = model(inputs)
    # calculate the loss and prediction performance statistics
    if type(output) == tuple:
        output, _ = output
        _, preds = torch.max(output.data, 1)
        print(preds)
        loss = loss_fun(output, labels)
        running_loss += loss
        running_corrects += preds.eq(labels.view_as(preds)).sum()
    # update dataset size
    size += batch_count

100%|██████████| 1350/1350 [02:27<00:00,  9.16it/s]


In [32]:
running_corrects

0